# imports and stuff

In [22]:
# Import libraries
import pandas as pd
import numpy as np
import mlflow
import os
from scipy import stats

# Set up MLflow tracking with DagsHub
os.environ["MLFLOW_TRACKING_URI"] = "https://dagshub.com/qetibakh/ML-Homework1.mlflow/"
os.environ["MLFLOW_TRACKING_USERNAME"] = "qetibakh"
os.environ["MLFLOW_TRACKING_PASSWORD"] = "5d5729711347a10a69c47eea029df715cc67e1ce"

# Load test data
test_data = pd.read_csv("data/test.csv")
print(f"Test data shape: {test_data.shape}")

# We also need the training data for reference
train_data = pd.read_csv("data/train.csv")
print(f"Training data shape: {train_data.shape}")

# Load the best model from MLflow Model Registry
model_name = "house_price_xgboost"
model_version = "2"  # Use the latest version (version 2 based on your output)
model_uri = f"models:/{model_name}/{model_version}"

print(f"Loading model from {model_uri}")
model = mlflow.pyfunc.load_model(model_uri)

# Start preprocessing - reproduce the same steps from model_experiment.ipynb
print("Starting data preprocessing...")
# Save original copies for reference
train_original = train_data.copy()
test_original = test_data.copy()

Test data shape: (1459, 80)
Training data shape: (1460, 81)
Loading model from models:/house_price_xgboost/2
Starting data preprocessing...


## cleaning

In [23]:
# Start MLflow run for cleaning
if mlflow.active_run():
    mlflow.end_run()
mlflow.start_run(run_name="data_cleaning")

# Function to log cleaning steps
def log_cleaning_step(description, affected_columns, technique):
    mlflow.log_param(f"cleaning_step_{description}_columns", str(affected_columns))
    mlflow.log_param(f"cleaning_step_{description}_technique", technique)

# Combine train and test for preprocessing
all_data = pd.concat([train_data.drop('SalePrice', axis=1), test_data])
print(f"Combined data shape: {all_data.shape}")

# Identify numerical and categorical features
num_features = all_data.select_dtypes(include=[np.number]).columns
cat_features = all_data.select_dtypes(include=['object']).columns

# Handle missing numerical data
numerical_na_cols = all_data[num_features].isnull().sum()[all_data[num_features].isnull().sum() > 0].index.tolist()
print(f"Numerical features with missing values: {numerical_na_cols}")

for col in numerical_na_cols:
    # Custom handling for each column with missing values
    if "garage" in col.lower() or "bsmt" in col.lower():
        # These are likely 0 if missing (no garage/basement)
        all_data[col] = all_data[col].fillna(0)
        log_cleaning_step(f"fill_na_{col}", col, "fill_with_zero_as_feature_absent")
    else:
        # Fill with median for other numerical features
        all_data[col] = all_data[col].fillna(0)
        log_cleaning_step(f"fill_na_{col}", col, "fill_with_median")

# Handle missing categorical data
categorical_na_cols = all_data[cat_features].isnull().sum()[all_data[cat_features].isnull().sum() > 0].index.tolist()
print(f"Categorical features with missing values: {categorical_na_cols}")

for col in categorical_na_cols:
    if col in ["Alley", "PoolQC", "Fence", "MiscFeature"]:
        # These are often NA because the feature doesn't exist
        all_data[col].fillna("None", inplace=True)
        log_cleaning_step(f"fill_na_{col}", col, "fill_with_None_as_feature_absent")
    elif "garage" in col.lower() or "bsmt" in col.lower():
        # If these are missing, likely no garage/basement
        all_data[col].fillna("None", inplace=True)
        log_cleaning_step(f"fill_na_{col}", col, "fill_with_None_as_feature_absent")
    else:
        # For other categorical features, use mode
        all_data[col].fillna(all_data[col].mode()[0], inplace=True)
        log_cleaning_step(f"fill_na_{col}", col, "fill_with_mode")

# Check for remaining missing values
missing_after = all_data.isnull().sum().sum()
if missing_after > 0:
    print("Columns with remaining missing values:")
    print(all_data.isnull().sum()[all_data.isnull().sum() > 0])

    # Fill any remaining NaN values with appropriate values
    # For numeric columns, use median
    num_cols = all_data.select_dtypes(include=[np.number]).columns
    for col in num_cols:
        if all_data[col].isnull().sum() > 0:
            all_data[col] = all_data[col].fillna(all_data[col].median())
            print(f"Filled NaN values in numeric column {col} with median")

    # For categorical columns, use mode
    cat_cols = all_data.select_dtypes(include=['object']).columns
    for col in cat_cols:
        if all_data[col].isnull().sum() > 0:
            all_data[col] = all_data[col].fillna(all_data[col].mode()[0])
            print(f"Filled NaN values in categorical column {col} with mode")

# Final verification
assert all_data.isnull().sum().sum() == 0, "There are still missing values after cleaning!"
print("All missing values have been handled successfully.")

# End cleaning MLflow run
mlflow.end_run()

Combined data shape: (2919, 80)
Numerical features with missing values: ['LotFrontage', 'MasVnrArea', 'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', 'BsmtFullBath', 'BsmtHalfBath', 'GarageYrBlt', 'GarageCars', 'GarageArea']
Categorical features with missing values: ['MSZoning', 'Alley', 'Utilities', 'Exterior1st', 'Exterior2nd', 'MasVnrType', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2', 'Electrical', 'KitchenQual', 'Functional', 'FireplaceQu', 'GarageType', 'GarageFinish', 'GarageQual', 'GarageCond', 'PoolQC', 'Fence', 'MiscFeature', 'SaleType']


C:\Users\Qeto\AppData\Local\Temp\ipykernel_8716\1601012655.py:50: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  all_data[col].fillna(all_data[col].mode()[0], inplace=True)
C:\Users\Qeto\AppData\Local\Temp\ipykernel_8716\1601012655.py:42: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

All missing values have been handled successfully.
🏃 View run data_cleaning at: https://dagshub.com/qetibakh/ML-Homework1.mlflow/#/experiments/0/runs/b3110ee21d2a45b8b5a88b06d87c681e
🧪 View experiment at: https://dagshub.com/qetibakh/ML-Homework1.mlflow/#/experiments/0


## feature engineering

In [24]:
# Start MLflow run for feature engineering
if mlflow.active_run():
    mlflow.end_run()
mlflow.start_run(run_name="feature_engineering")

# Function to log feature engineering steps
def log_feature_eng_step(description, technique, affected_columns=None, created_columns=None):
    if affected_columns:
        mlflow.log_param(f"fe_step_{description}_affected", str(affected_columns))
    if created_columns:
        mlflow.log_param(f"fe_step_{description}_created", str(created_columns))
    mlflow.log_param(f"fe_step_{description}_technique", technique)

# Transform skewed numerical features
numeric_features = all_data.select_dtypes(include=[np.number]).columns

# Calculate skewness for each numerical feature
from scipy import stats
skewed_features = all_data[numeric_features].apply(lambda x: stats.skew(x.dropna())).sort_values(ascending=False)
high_skew_features = skewed_features[skewed_features > 0.5].index

print(f"Number of highly skewed features: {len(high_skew_features)}")
mlflow.log_param("num_skewed_features", len(high_skew_features))

# Apply log transformation to highly skewed features
for feature in high_skew_features:
    # Adding 1 to avoid log(0)
    all_data[feature] = np.log1p(all_data[feature])

log_feature_eng_step("log_transform_skewed", "log1p_transform", affected_columns=high_skew_features.tolist())

# Encode categorical variables
# Ordinal encoding for quality and condition features
ordinal_quality_mapping = {
    'Po': 1, 'Fa': 2, 'TA': 3, 'Gd': 4, 'Ex': 5, 'None': 0, 'NA': 0
}

quality_cols = [col for col in all_data.columns if 'Qual' in col or 'Cond' in col]
quality_cols = [col for col in quality_cols if col in cat_features]

for col in quality_cols:
    all_data[col + '_Encoded'] = all_data[col].map(ordinal_quality_mapping)

log_feature_eng_step("ordinal_encoding_quality", "ordinal_mapping",
                    affected_columns=quality_cols,
                    created_columns=[col + '_Encoded' for col in quality_cols])

# One-hot encoding for other categorical features
categorical_cols = [col for col in cat_features if col not in quality_cols]
print(f"Number of categorical columns to one-hot encode: {len(categorical_cols)}")

# Apply one-hot encoding
all_data_encoded = pd.get_dummies(all_data, columns=categorical_cols, drop_first=True)
print(f"Shape after one-hot encoding: {all_data_encoded.shape}")

log_feature_eng_step("one_hot_encoding", "pd_get_dummies",
                    affected_columns=categorical_cols,
                    created_columns=[col for col in all_data_encoded.columns if col not in all_data.columns])

# Create new features
# Total square footage
all_data_encoded['TotalSF'] = all_data_encoded['1stFlrSF'] + all_data_encoded['2ndFlrSF'] + all_data_encoded['TotalBsmtSF']
log_feature_eng_step("total_square_feet", "sum_areas",
                    affected_columns=['1stFlrSF', '2ndFlrSF', 'TotalBsmtSF'],
                    created_columns=['TotalSF'])

# Total bathrooms
all_data_encoded['TotalBath'] = all_data_encoded['FullBath'] + 0.5 * all_data_encoded['HalfBath'] + \
                               all_data_encoded['BsmtFullBath'] + 0.5 * all_data_encoded['BsmtHalfBath']
log_feature_eng_step("total_bathrooms", "weighted_sum",
                    affected_columns=['FullBath', 'HalfBath', 'BsmtFullBath', 'BsmtHalfBath'],
                    created_columns=['TotalBath'])

# House age and when it was remodeled
all_data_encoded['Age'] = all_data_encoded['YrSold'] - all_data_encoded['YearBuilt']
all_data_encoded['Remodeled'] = (all_data_encoded['YearRemodAdd'] != all_data_encoded['YearBuilt']).astype(int)
all_data_encoded['RemodAge'] = all_data_encoded['YrSold'] - all_data_encoded['YearRemodAdd']
log_feature_eng_step("age_features", "year_differences",
                    affected_columns=['YrSold', 'YearBuilt', 'YearRemodAdd'],
                    created_columns=['Age', 'Remodeled', 'RemodAge'])

# Log features created
mlflow.log_param("num_features_before", all_data.shape[1])
mlflow.log_param("num_features_after", all_data_encoded.shape[1])
mlflow.log_param("new_features_created", all_data_encoded.shape[1] - all_data.shape[1])

# End feature engineering MLflow run
mlflow.end_run()

Number of highly skewed features: 25
Number of categorical columns to one-hot encode: 33
Shape after one-hot encoding: (2919, 230)
🏃 View run feature_engineering at: https://dagshub.com/qetibakh/ML-Homework1.mlflow/#/experiments/0/runs/65955a5ecbea4c43adf5b03bfd1fe0f4
🧪 View experiment at: https://dagshub.com/qetibakh/ML-Homework1.mlflow/#/experiments/0


## feature selection

In [25]:
# Start MLflow run for feature selection
if mlflow.active_run():
    mlflow.end_run()
mlflow.start_run(run_name="feature_selection")

# Let's first check what's happening with the indices
print(f"Original train data shape: {train_original.shape}")
print(f"Original train index range: {min(train_original.index)} to {max(train_original.index)}")
print(f"all_data_encoded shape: {all_data_encoded.shape}")
print(f"all_data_encoded index range: {min(all_data_encoded.index)} to {max(all_data_encoded.index)}")

# We need to understand the difference between train_original.index and train_idx
train_idx = train_original.index

# A better approach - use integer positions since we know train data comes first
train_rows = train_original.shape[0]
X_train_full = all_data_encoded.iloc[:train_rows].copy()
X_test = all_data_encoded.iloc[train_rows:].copy()
y_train = np.log1p(train_original['SalePrice'])  # Log transform for better model performance

# Verify the shapes are correct
print(f"X_train_full shape: {X_train_full.shape}")
print(f"y_train shape: {y_train.shape}")
print(f"X_test shape: {X_test.shape}")
assert X_train_full.shape[0] == y_train.shape[0], "X and y have different number of samples!"

# Check for categorical columns that might cause issues
categorical_columns = X_train_full.select_dtypes(include=['object']).columns
if len(categorical_columns) > 0:
    print(f"Found categorical columns that need handling: {categorical_columns.tolist()}")
    # Create a numeric-only version of the data
    X_train_full_numeric = X_train_full.select_dtypes(include=[np.number])
    print(f"Using {X_train_full_numeric.shape[1]} numeric features for all feature selection methods")
else:
    X_train_full_numeric = X_train_full
    print("All features are already numeric")

# Check for and handle NaN values
if X_train_full_numeric.isnull().values.any():
    print(f"Found NaN values in numeric data.")
    # Count NaN values by column
    nan_counts = X_train_full_numeric.isnull().sum()
    columns_with_nan = nan_counts[nan_counts > 0]
    print(f"Columns with NaN values:\n{columns_with_nan}")

    # Identify columns that are entirely NaN
    all_nan_columns = [col for col in X_train_full_numeric.columns
                      if X_train_full_numeric[col].isnull().all()]

    if all_nan_columns:
        print(f"These columns are entirely NaN and will be dropped: {all_nan_columns}")
        X_train_full_numeric = X_train_full_numeric.drop(columns=all_nan_columns)

    # For the remaining columns with some NaN values, fill with median
    for col in X_train_full_numeric.columns:
        if X_train_full_numeric[col].isnull().any():
            X_train_full_numeric[col] = X_train_full_numeric[col].fillna(X_train_full_numeric[col].median())
            print(f"Filled NaN values in column {col} with median")

    # Verify no NaN values remain
    assert not X_train_full_numeric.isnull().values.any(), "NaN values still present after filling"
    print("All NaN values have been handled")

# Correlation with target variable
correlation_method = {}

# Add SalePrice back for correlation calculation
X_train_with_target = X_train_full_numeric.copy()
X_train_with_target['SalePrice'] = train_original['SalePrice']

# Calculate correlation using only numeric data
correlation = X_train_with_target.corr()['SalePrice'].sort_values(ascending=False)

# Select features with absolute correlation > threshold
thresholds = [0.1, 0.2, 0.3]
for threshold in thresholds:
    corr_selected = correlation[abs(correlation) > threshold].index.tolist()
    corr_selected.remove('SalePrice')  # Remove target
    correlation_method[f"corr_{threshold}"] = corr_selected
    print(f"Selected {len(corr_selected)} features using correlation threshold {threshold}")
    mlflow.log_param(f"num_features_corr_{threshold}", len(corr_selected))

# Lasso feature selection
from sklearn.linear_model import Lasso
from sklearn.feature_selection import SelectFromModel

# Use numeric-only data for Lasso
lasso_selector = SelectFromModel(Lasso(alpha=0.005, random_state=42))
lasso_selector.fit(X_train_full_numeric, y_train)

lasso_selected = X_train_full_numeric.columns[lasso_selector.get_support()].tolist()
print(f"Selected {len(lasso_selected)} features using Lasso")
mlflow.log_param("num_features_lasso", len(lasso_selected))

# Random Forest feature importance
from sklearn.ensemble import RandomForestRegressor

# Use numeric-only data for Random Forest
rf = RandomForestRegressor(n_estimators=100, random_state=42)
rf.fit(X_train_full_numeric, y_train)

# Get feature importances
importances = rf.feature_importances_
indices = np.argsort(importances)[::-1]

# Select top features by importance
rf_selected_top30 = X_train_full_numeric.columns[indices[:min(30, len(indices))]].tolist()
rf_selected_top50 = X_train_full_numeric.columns[indices[:min(50, len(indices))]].tolist()

print(f"Selected top 30 features using Random Forest importance")
print(f"Selected top 50 features using Random Forest importance")
mlflow.log_param("num_features_rf_top30", len(rf_selected_top30))
mlflow.log_param("num_features_rf_top50", len(rf_selected_top50))

# Create different feature sets for experimentation
feature_sets = {
    'all_features_numeric': X_train_full_numeric.columns.tolist(),
    'corr_0.1': correlation_method['corr_0.1'],
    'corr_0.3': correlation_method['corr_0.3'],
    'lasso_selected': lasso_selected,
    'rf_top30': rf_selected_top30,
    'rf_top50': rf_selected_top50
}

# Log feature sets
for set_name, features in feature_sets.items():
    mlflow.log_param(f"feature_set_{set_name}_count", len(features))

# End feature selection MLflow run
mlflow.end_run()

Original train data shape: (1460, 81)
Original train index range: 0 to 1459
all_data_encoded shape: (2919, 235)
all_data_encoded index range: 0 to 1459
X_train_full shape: (1460, 235)
y_train shape: (1460,)
X_test shape: (1459, 235)
Found categorical columns that need handling: ['Condition1', 'Condition2', 'ExterQual', 'ExterCond', 'BsmtQual', 'BsmtCond', 'KitchenQual', 'GarageQual', 'GarageCond', 'SaleCondition']
Using 52 numeric features for all feature selection methods
Found NaN values in numeric data.
Columns with NaN values:
Condition1_Encoded       1460
Condition2_Encoded       1460
SaleCondition_Encoded    1460
dtype: int64
These columns are entirely NaN and will be dropped: ['Condition1_Encoded', 'Condition2_Encoded', 'SaleCondition_Encoded']
All NaN values have been handled
Selected 35 features using correlation threshold 0.1
Selected 29 features using correlation threshold 0.2
Selected 22 features using correlation threshold 0.3
Selected 24 features using Lasso
Selected top 

## submission file

In [26]:
print("Making predictions...")
# Extract test data from the processed dataset
test_data_processed = X_test.copy()

# Check for NaN values in preprocessed test data
if test_data_processed.isnull().values.any():
    print("Warning: NaN values found in preprocessed test data")
    print(test_data_processed.isnull().sum()[test_data_processed.isnull().sum() > 0])
    # Fill NaN values with 0
    test_data_processed = test_data_processed.fillna(0)
    print("Filled NaN values with 0")

# Make predictions
predictions = model.predict(test_data_processed)

# Transform back from log space
predictions = np.expm1(predictions)

# Create submission file
submission = pd.DataFrame({
    'Id': test_data['Id'],
    'SalePrice': predictions
})

# Save submission file
submission.to_csv('submission.csv', index=False)
print("Submission file created successfully!")

# Display first 5 predictions
print("\nSample predictions:")
print(submission.head())

2025/04/10 18:45:54 WARNING mlflow.models.utils: Found extra inputs in the model input that are not defined in the model signature: `['MiscFeature_TenC', 'Exterior2nd_Wd Shng', 'LotShape_IR2', 'Fence_GdWo', 'HouseStyle_2.5Unf', 'GarageFinish_None', 'Exterior2nd_BrkFace', 'BsmtFinType1_None', 'LandContour_HLS', 'BsmtFinType1_Rec', 'BldgType_Twnhs', 'Electrical_Mix', 'Exterior2nd_Wd Sdng', 'Heating_Grav', 'PavedDrive_P', 'MSZoning_RH', 'Foundation_Stone', 'BsmtExposure_Gd', 'BsmtFinType2_None', 'GarageCond', 'LandSlope_Mod', 'Electrical_FuseP', 'RoofStyle_Gambrel', 'GarageType_BuiltIn', 'HeatingQC_Po', 'Neighborhood_Sawyer', 'Exterior2nd_MetalSd', 'GarageFinish_Unf', 'BsmtFinType2_Unf', 'Neighborhood_NAmes', 'GarageType_Detchd', 'MSZoning_FV', 'BsmtExposure_Mn', 'BsmtFinType2_GLQ', 'LandContour_Lvl', 'Exterior2nd_ImStucc', 'SaleType_CWD', 'BldgType_2fmCon', 'MiscFeature_Shed', 'BsmtFinType1_BLQ', 'Exterior1st_Stucco', 'Functional_Sev', 'MiscFeature_Othr', 'Exterior1st_BrkComm', 'Heating_

Making predictions...
Condition1_Encoded       1459
Condition2_Encoded       1459
SaleCondition_Encoded    1459
dtype: int64
Filled NaN values with 0
Submission file created successfully!

Sample predictions:
     Id      SalePrice
0  1461  131459.640625
1  1462  155973.375000
2  1463  187002.140625
3  1464  189090.859375
4  1465  186770.625000
